<a href="https://colab.research.google.com/github/meozbrls/Flight_Prices_Analysis/blob/buse/notebooks/Analysis%26KPI/price_dynamics_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Flight Price Dynamics Analysis**

### **Environment Setup and Library Imports**

In this section, we install the necessary libraries for data handling and drive access. We also set up the local directory structure to organize our raw data and processed outputs.

In [1]:
# Install required libraries for data downloading and parquet support
!pip -q install gdown pyarrow

import gdown, os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# Create directories for data management
os.makedirs("data", exist_ok=True)
os.makedirs("out", exist_ok=True)
print("Environment Ready: Directories created.")

Environment Ready: Directories created.


### **Data Acquisition**

We are fetching the raw flight price dataset from a remote Google Drive source using its file ID and saving it locally.

In [2]:
# Define Google Drive File ID and target path
FILE_ID = "1TrX_MuNS-EvrjKutCqzLnOa2W6X6Y3Dm"
OUTPUT_PATH = "data/flightprices.csv"

# Download the file
url = f"https://drive.google.com/uc?id={FILE_ID}"
gdown.download(url, OUTPUT_PATH, quiet=False)
print("File successfully downloaded to:", OUTPUT_PATH)

Downloading...
From (original): https://drive.google.com/uc?id=1TrX_MuNS-EvrjKutCqzLnOa2W6X6Y3Dm
From (redirected): https://drive.google.com/uc?id=1TrX_MuNS-EvrjKutCqzLnOa2W6X6Y3Dm&confirm=t&uuid=9d6a1a21-521e-4d0b-a8e9-d6a46ed83809
To: /content/data/flightprices.csv
100%|██████████| 2.63G/2.63G [00:24<00:00, 106MB/s]


File successfully downloaded to: data/flightprices.csv


In [3]:
import os
os.listdir("data")


['price_time_core.csv', 'flightprices.csv']

Since flight datasets can be extremely large, we process the file in "chunks" to avoid memory (RAM) issues. We filter for only the columns we need for our analysis.

In [4]:
# Define columns to keep for analysis to optimize memory usage
USE_COLS = [
    "legId", "searchDate", "flightDate", "startingAirport",
    "destinationAirport", "baseFare", "totalFare", "isNonStop",
    "isBasicEconomy", "seatsRemaining", "segmentsAirlineName",
    "segmentsAirlineCode", "travelDuration"
]

INPUT_CSV = "data/flightprices.csv"
OUTPUT_CSV = "data/price_time_core.csv"
CHUNK_SIZE = 300_000

# Clear existing output file if it exists
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

# Read and write in chunks
chunks = pd.read_csv(INPUT_CSV, usecols=USE_COLS, chunksize=CHUNK_SIZE, low_memory=False)

for i, chunk in enumerate(chunks):
    chunk.to_csv(OUTPUT_CSV, index=False, mode="a", header=(i==0))
    # Log progress for every chunk processed
    print(f"Chunk {i+1} is writen: {chunk.shape}")

print("New optimized CSV created:", OUTPUT_CSV)

Chunk 1 is writen: (300000, 13)
Chunk 2 is writen: (300000, 13)
Chunk 3 is writen: (300000, 13)
Chunk 4 is writen: (300000, 13)
Chunk 5 is writen: (300000, 13)
Chunk 6 is writen: (300000, 13)
Chunk 7 is writen: (300000, 13)
Chunk 8 is writen: (300000, 13)
Chunk 9 is writen: (300000, 13)
Chunk 10 is writen: (300000, 13)
Chunk 11 is writen: (300000, 13)
Chunk 12 is writen: (300000, 13)
Chunk 13 is writen: (300000, 13)
Chunk 14 is writen: (300000, 13)
Chunk 15 is writen: (300000, 13)
Chunk 16 is writen: (300000, 13)
Chunk 17 is writen: (300000, 13)
Chunk 18 is writen: (300000, 13)
Chunk 19 is writen: (300000, 13)
Chunk 20 is writen: (300000, 13)
Chunk 21 is writen: (300000, 13)
Chunk 22 is writen: (300000, 13)
Chunk 23 is writen: (300000, 13)
Chunk 24 is writen: (300000, 13)
Chunk 25 is writen: (300000, 13)
Chunk 26 is writen: (300000, 13)
Chunk 27 is writen: (300000, 13)
Chunk 28 is writen: (300000, 13)
Chunk 29 is writen: (300000, 13)
Chunk 30 is writen: (300000, 13)
Chunk 31 is writen:

### **Feature Engineering**

In this step, we transform raw data into meaningful insights. We calculate the "days before flight" (lead time), group these into "buckets" for better visualization, and extract airline names.

In [5]:
# Load the optimized dataset
df = pd.read_csv("data/price_time_core.csv", low_memory=False)
print(df.shape)
df.head()

(11774933, 13)


,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isNonStop,baseFare,totalFare,seatsRemaining,segmentsAirlineName,segmentsAirlineCode
0,684e879e0f2cefcdbfba1f7e35f897a3,2022-04-17,2022-06-01,ATL,BOS,PT2H28M,False,True,31.00,87.59,0,Spirit Airlines,NK
1,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,ATL,BOS,PT4H50M,False,False,65.48,93.99,4,Frontier Airlines||Frontier Airlines,F9||F9
2,5b13b222dff8d227c34ba5f0b10a8b5a,2022-04-17,2022-06-01,ATL,BOS,PT8H38M,False,False,25.58,95.16,0,Spirit Airlines||Spirit Airlines,NK||NK
3,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,ATL,BOS,PT2H37M,False,True,161.86,188.60,9,Delta,DL
4,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,ATL,BOS,PT2H41M,False,True,161.86,188.60,2,Delta,DL


In [6]:
import pandas as pd
import numpy as np

# Convert dates to datetime objects
df["searchDate"] = pd.to_datetime(df["searchDate"], errors="coerce")
df["flightDate"] = pd.to_datetime(df["flightDate"], errors="coerce")

# Clean missing dates
df = df.dropna(subset=["searchDate", "flightDate"]).copy()

# Calculate days before flight
df["days_before_flight"] = (df["flightDate"] - df["searchDate"]).dt.days
df = df[df["days_before_flight"] >= 0].copy() # Filter out data errors

# Create flight route feature
df["route"] = df["startingAirport"].astype(str) + "-" + df["destinationAirport"].astype(str)

# Create flight month (YYYY-MM)
df["flight_month"] = df["flightDate"].dt.to_period("M").astype(str)

# Create lead time buckets for categorical analysis
bins = [-1, 3, 7, 14, 30, 60, 10**9]
labels = ["0-3", "4-7", "8-14", "15-30", "31-60", "61+"]
df["lead_time_bucket"] = pd.cut(df["days_before_flight"], bins=bins, labels=labels)

# Clean airline names (extracting the primary carrier)
df["airline"] = df["segmentsAirlineName"].astype(str).str.split(r"\|\|").str[0]
df["airline_code"] = df["segmentsAirlineCode"].astype(str).str.split(r"\|\|").str[0]

# Final price cleaning
df["totalFare"] = pd.to_numeric(df["totalFare"], errors="coerce")
df["baseFare"] = pd.to_numeric(df["baseFare"], errors="coerce")
df = df.dropna(subset=["totalFare"]).copy()
df = df[df["totalFare"]>0].copy()

print("Feature engineering complete. Final shape:", df.shape)
df[["legId","searchDate","flightDate","days_before_flight","lead_time_bucket","route","flight_month","airline","totalFare"]].head()


Feature engineering complete. Final shape: (11774933, 19)


,legId,searchDate,flightDate,days_before_flight,lead_time_bucket,route,flight_month,airline,totalFare
0,684e879e0f2cefcdbfba1f7e35f897a3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,2022-06,Spirit Airlines,87.59
1,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,2022-06,Frontier Airlines,93.99
2,5b13b222dff8d227c34ba5f0b10a8b5a,2022-04-17,2022-06-01,45,31-60,ATL-BOS,2022-06,Spirit Airlines,95.16
3,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,45,31-60,ATL-BOS,2022-06,Delta,188.60
4,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,45,31-60,ATL-BOS,2022-06,Delta,188.60


### **Global Timing and Price Aggregation**

We analyze how flight prices change based on how many days in advance the ticket is searched. This provides a "Global Price Index" based on booking lead time.

In [29]:
df["lead_time_bucket"].value_counts()


,count
lead_time_bucket,
31-60,4935909
15-30,3153881
8-14,1827298
4-7,1052013
0-3,805832
61+,0


In [8]:
price_df = df[
    [
        "legId",
        "searchDate",
        "flightDate",
        "days_before_flight",
        "lead_time_bucket",
        "route",
        "airline",
        "totalFare"
    ]
].copy()

price_df.head()


,legId,searchDate,flightDate,days_before_flight,lead_time_bucket,route,airline,totalFare
0,684e879e0f2cefcdbfba1f7e35f897a3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Spirit Airlines,87.59
1,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Frontier Airlines,93.99
2,5b13b222dff8d227c34ba5f0b10a8b5a,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Spirit Airlines,95.16
3,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Delta,188.60
4,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Delta,188.60


In [9]:
price_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11774933 entries, 0 to 11774932
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   legId               object        
 1   searchDate          datetime64[ns]
 2   flightDate          datetime64[ns]
 3   days_before_flight  int64         
 4   lead_time_bucket    category      
 5   route               object        
 6   airline             object        
 7   totalFare           float64       
dtypes: category(1), datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 640.1+ MB


In [10]:
# Aggregate pricing metrics by lead time bucket
global_timing = (
    price_df.groupby("lead_time_bucket")
    .agg(
        flight_count=("legId", "count"),
        avg_price=("totalFare", "mean"),
        median_price=("totalFare", "median"),
        min_price=("totalFare", "min"),
        max_price=("totalFare", "max"),
        std_price=("totalFare", "std")
    )
    .reset_index()
)

# Set the logical order for the buckets
bucket_order = ["0-3", "4-7", "8-14", "15-30", "31-60"]
global_timing = global_timing[global_timing["lead_time_bucket"] != "61+"].copy()
global_timing.sort_values("lead_time_bucket", inplace=True)

global_timing


/tmp/ipython-input-3838345151.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_df.groupby("lead_time_bucket")


,lead_time_bucket,flight_count,avg_price,median_price,min_price,max_price,std_price
0,0-3,805832,400.413046,358.6,30.69,4752.60,227.367352
1,4-7,1052013,399.666675,362.2,29.59,4752.60,220.848119
2,8-14,1827298,402.781791,371.6,29.59,7554.20,224.023025
3,15-30,3153881,410.705152,381.6,30.69,8260.61,226.562836
4,31-60,4935909,392.519788,365.6,30.69,8260.61,209.537686


**Data Cleaning and Price Trend Validation**

To ensure a clean analysis, we remove buckets with insufficient or null data (like the 61+ window). Following the cleaning, we perform logical checks to validate common industry assumptions: the Last Minute Price Increases and the Early Reservation Advantage.

In [11]:
global_timing = global_timing[global_timing["lead_time_bucket"] != "61+"].copy()

In [12]:
# Filter main dataframe to include only the relevant booking windows
bucket_order = ["0-3", "4-7", "8-14", "15-30", "31-60"]
price_df = price_df[price_df["lead_time_bucket"].isin(bucket_order)].copy()

In [13]:
# Extract median prices for quick logical validation
gt = global_timing.set_index("lead_time_bucket")["median_price"]

In [14]:
# Test 1: Last Minute Price Increase
# Checking if the median price for 0-3 days is higher than the 8-14 days window
is_last_minute_expensive = gt.loc["0-3"] > gt.loc["8-14"]
print(f"Is there a last minute price hike? {is_last_minute_expensive}")

Is there a last minute price hike? False


In [15]:
# Test 2: Early Reservation Advantage
# Checking if booking 15-30 days in advance is cheaper than booking 8-14 days in advance
is_early_booking_cheaper = gt.loc["15-30"] < gt.loc["8-14"]
print(f"Is early reservation more advantageous? {is_early_booking_cheaper}")

Is early reservation more advantageous? False


## **Route-Specific Price Dynamics**

In this section, we transition from a global view to a route-specific analysis. By grouping data by airport pairs (Routes), we can identify which specific journeys are more sensitive to booking time.

In [16]:
# Aggregate pricing metrics by Route and Lead Time
route_timing = (
    price_df
    .groupby(["route", "lead_time_bucket"], observed=True)
    .agg(
        flight_count=("legId","count"),
        avg_price=("totalFare","mean"),
        median_price=("totalFare","median"),
        min_price=("totalFare","min"),
        max_price=("totalFare", "max"),
        std_price=("totalFare","std"),
    )
    .reset_index()
)

# Ensure the lead time categories follow a logical chronological order
route_timing["lead_time_bucket"] = pd.Categorical(
    route_timing["lead_time_bucket"],
    categories=bucket_order,
    ordered=True
)

route_timing = route_timing.sort_values(["route","lead_time_bucket"])
route_timing.head(10)

,route,lead_time_bucket,flight_count,avg_price,median_price,min_price,max_price,std_price
0,ATL-BOS,0-3,5184,371.121329,360.1,136.10,1487.61,134.562860
1,ATL-BOS,4-7,6398,353.742082,342.2,67.88,1324.10,122.821392
2,ATL-BOS,8-14,10738,337.233551,326.6,55.78,1717.60,120.427748
3,ATL-BOS,15-30,18384,311.601155,298.6,55.78,1324.10,108.699952
4,ATL-BOS,31-60,27111,314.031774,297.2,55.88,1124.60,113.761978
5,ATL-CLT,0-3,4198,429.624631,408.6,242.20,1881.60,72.269366
6,ATL-CLT,4-7,4875,405.608355,398.1,240.70,1087.10,65.677148
7,ATL-CLT,8-14,6946,368.588534,358.6,198.60,977.10,74.473199
8,ATL-CLT,15-30,8543,274.811147,258.6,148.60,1921.60,105.368745
9,ATL-CLT,31-60,12855,261.706222,233.6,148.60,856.64,104.159884


### **Last-Minute Price Increase & Early Reservation Analysis**

To quantify the impact of booking late, we calculate the percentage price increase between different windows. Specifically, we compare the "Last Minute" (0-3 days) window against the "Standard" (8-14 days)

window to find the routes with the highest price volatility.

In [17]:
# Pivot the table to compare price buckets side-by-side per route
lead_time_price_pct = route_timing.pivot(
    index="route",
    columns="lead_time_bucket",
    values="median_price"
).reset_index()

# Calculate the percentage increase for last-minute bookings
lead_time_price_pct["last_minute_price_increase_pct"] = (
    (lead_time_price_pct["0-3"] / lead_time_price_pct["8-14"] - 1) * 100
)

# Calculate the savings potential: Early (31-60 days) vs. Late (0-3 days)
lead_time_price_pct["early_vs_late_pct"] = (
    (lead_time_price_pct["31-60"] / lead_time_price_pct["0-3"] - 1) * 100
)

# Display the top 10 routes with the highest last-minute price spikes
lead_time_price_pct.sort_values("last_minute_price_increase_pct", ascending=False).head(10)

lead_time_bucket,route,0-3,4-7,8-14,15-30,31-60,last_minute_price_increase_pct,early_vs_late_pct
20,BOS-EWR,163.60,122.58,108.60,98.600,148.60,50.644567,-9.168704
79,DTW-DFW,498.60,417.20,338.60,337.610,313.60,47.253396,-37.103891
210,PHL-DTW,508.60,376.10,356.59,278.600,260.58,42.628789,-48.765238
64,DFW-DTW,498.60,429.60,349.59,342.105,304.60,42.624217,-38.908945
165,MIA-DTW,306.60,294.60,217.61,247.610,247.61,40.894260,-19.240052
85,DTW-MIA,305.61,294.60,217.61,257.600,247.61,40.439318,-18.978437
213,PHL-JFK,391.59,321.60,281.60,264.600,242.20,39.058949,-38.149595
127,JFK-MIA,206.60,174.60,148.60,198.600,216.11,39.030956,4.603098
88,DTW-PHL,508.60,377.60,366.10,288.600,282.20,38.923791,-44.514353
91,EWR-BOS,163.60,135.58,118.60,118.600,158.60,37.942664,-3.056235


### **Determining the Optimal Booking Window per Route**

Identifies the single best lead-time bucket for every route based on the lowest median price.

In [18]:
# Identify the cheapest lead-time window for each specific route
optimal_bucket = (
    route_timing
    .dropna(subset=["median_price", "lead_time_bucket"])
    .sort_values(["route", "median_price"])
    .groupby("route", as_index=False)
    .first()
)

# Rename columns for clarity in final reports
optimal_bucket = optimal_bucket.rename(columns={
    "lead_time_bucket": "cheapest_lead_time_bucket",
    "median_price": "cheapest_median_price"
})

optimal_bucket.head()

,route,cheapest_lead_time_bucket,flight_count,avg_price,cheapest_median_price,min_price,max_price,std_price
0,ATL-BOS,31-60,27111,314.031774,297.2,55.88,1124.60,113.761978
1,ATL-CLT,31-60,12855,261.706222,233.6,148.60,856.64,104.159884
2,ATL-DEN,0-3,2770,389.758238,367.2,133.98,1514.60,136.693616
3,ATL-DFW,0-3,2927,292.935453,244.6,123.98,2039.60,131.465271
4,ATL-DTW,31-60,19151,255.489169,232.1,42.79,988.60,113.192418


### **Daily Search Trends and Statistical Significance**

Investigate whether the day of the week you search for a flight affects the price. We use the Kruskal-Wallis H-test to determine if there is a statistically significant difference in prices between different search days.

In [19]:
# Extracting the day of the week from search dates
price_df["search_day"] = price_df["searchDate"].dt.day_name()
price_df["search_dow"] = price_df["searchDate"].dt.weekday  # 0=Monday
price_df["is_weekend_search"] = price_df["search_dow"].isin([5, 6])
price_df.head()

,legId,searchDate,flightDate,days_before_flight,lead_time_bucket,route,airline,totalFare,search_day,search_dow,is_weekend_search
0,684e879e0f2cefcdbfba1f7e35f897a3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Spirit Airlines,87.59,Sunday,6,True
1,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Frontier Airlines,93.99,Sunday,6,True
2,5b13b222dff8d227c34ba5f0b10a8b5a,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Spirit Airlines,95.16,Sunday,6,True
3,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Delta,188.60,Sunday,6,True
4,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,45,31-60,ATL-BOS,Delta,188.60,Sunday,6,True


In [20]:
# Statistical Test: Is the price difference between days significant?
from scipy.stats import kruskal

groups = [g["totalFare"].values for _, g in price_df.groupby("search_day")]
stat, p_val = kruskal(*groups)
print(f"Kruskal-Wallis Test Result: Statistics={stat:.2f}, p-value={p_val:.4f}")

Kruskal-Wallis Test Result: Statistics=5345.75, p-value=0.0000


### **Testing the "Early Week" Discount Hypothesis**

Perform a Mann-Whitney U test to statistically validate if searching on Monday and Tuesday actually yields lower prices compared to the rest of the week. This helps confirm if "Early Week" is the optimal search window.

In [21]:
from scipy.stats import mannwhitneyu

# Grouping prices: Early Week (Mon-Tue) vs. Others
mt_prices = price_df[price_df["search_day"].isin(["Monday", "Tuesday"])]["totalFare"]
other_prices = price_df[~price_df["search_day"].isin(["Monday", "Tuesday"])]["totalFare"]

# Hypothesis Test: Are Monday & Tuesday prices significantly lower?
u_stat, p_value = mannwhitneyu(mt_prices, other_prices, alternative="less")

# Interpretation
if p_value < 0.05:
    print("Result: Reject H0. Monday & Tuesday are statistically cheaper search days.")
else:
    print("Result: Fail to reject H0. No significant price difference found for Mon/Tue.")

Result: Reject H0. Monday & Tuesday are statistically cheaper search days.


We calculate key metrics such as median price and standard deviation for each day to identify potential "cheap search windows."

In [22]:
# Grouping by search day to see general price trends
dow_stats = (
    price_df.groupby("search_day")
      .agg(
          flight_count=("totalFare", "count"),
          avg_price=("totalFare", "mean"),
          median_price=("totalFare", "median"),
          std_price=("totalFare", "std")
      )
      .reset_index()
)

# Sorting by logical day order for better visualization
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
dow_stats["search_day"] = pd.Categorical(dow_stats["search_day"], categories=order, ordered=True)
dow_stats = dow_stats.sort_values("search_day")

dow_stats

,search_day,flight_count,avg_price,median_price,std_price
1,Monday,1571281,392.288649,361.60,213.375449
5,Tuesday,1145409,397.074839,366.70,223.066745
6,Wednesday,1565325,407.422063,377.60,219.774403
4,Thursday,2064489,403.798369,371.71,219.626824
0,Friday,1966143,399.434676,368.60,216.498812
2,Saturday,1785497,399.099366,368.60,216.979468
3,Sunday,1676789,400.377888,367.60,223.516910


### **Cross-Analysis of Search Day and Booking Lead Time**

We analyze the interaction between the search day and the lead time (how many days before the flight). This helps determine if the "cheapest day to book" changes depending on how close the departure date is.

In [23]:
# Analyzing the relationship between booking window (lead time) and search day
dow_lt = (
    price_df.groupby(["lead_time_bucket", "search_day"])
      .agg(
          flight_count=("totalFare", "count"),
          median_price=("totalFare", "median")
      )
      .reset_index()
)
dow_lt.head()

/tmp/ipython-input-2735490036.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_df.groupby(["lead_time_bucket", "search_day"])


,lead_time_bucket,search_day,flight_count,median_price
0,0-3,Friday,93742,387.10
1,0-3,Monday,137634,338.61
2,0-3,Saturday,103925,359.58
3,0-3,Sunday,125416,338.61
4,0-3,Thursday,97211,391.59


### **Route-Specific Day Trends**

Since every flight route has different market dynamics, we calculate the median prices for each specific route per day. This allows us to find the absolute "cheapest day" for every unique airport pair.

In [24]:
# Grouping by route and search day to identify route-specific trends
route_dow = (
    price_df.groupby(["route", "search_day"])
      .agg(
          flight_count=("totalFare", "count"),
          median_price=("totalFare", "median")
      )
      .reset_index()
)

In [25]:
# Identifying the absolute cheapest day of the week for every route
cheapest_day_per_route = (
    route_dow.sort_values("median_price")
             .groupby("route")
             .first()
             .reset_index()
)
cheapest_day_per_route.head()

,route,search_day,flight_count,median_price
0,ATL-BOS,Friday,11304,301.61
1,ATL-CLT,Saturday,5620,327.60
2,ATL-DEN,Tuesday,4314,408.60
3,ATL-DFW,Tuesday,4135,259.60
4,ATL-DTW,Friday,8566,287.20


### **Final Recommendation Engine (Route x Lead Time x Search Day)**

Combining the route, the booking window, and the search day, we pinpoint exactly when a user should book a flight for their specific destination to get the lowest possible price.

In [26]:
# Detailed aggregation for the final recommendation model
route_bucket_dow = (
    price_df.assign(
        search_day=price_df["searchDate"].dt.day_name(),
    )
    .groupby(["route","lead_time_bucket","search_day"], observed=True)
    .agg(
        flight_count=("totalFare","size"),
        median_price=("totalFare","median"),
        avg_price=("totalFare","mean"),
    )
    .reset_index()
)

# Selecting the best (cheapest) search day for every route and lead time combination
best_day = (
    route_bucket_dow.sort_values(["route","lead_time_bucket","median_price"])
    .groupby(["route","lead_time_bucket"], as_index=False)
    .first()
    .rename(columns={"search_day":"cheapest_search_day"})
)

best_day.head()

/tmp/ipython-input-311240894.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["route","lead_time_bucket"], as_index=False)


,route,lead_time_bucket,cheapest_search_day,flight_count,median_price,avg_price
0,ATL-BOS,0-3,Monday,857.0,318.6,350.659860
1,ATL-BOS,4-7,Thursday,1002.0,318.6,344.841327
2,ATL-BOS,8-14,Monday,1524.0,318.6,332.150407
3,ATL-BOS,15-30,Monday,1896.0,298.6,302.554852
4,ATL-BOS,31-60,Friday,4810.0,292.2,310.321871


### **Data Export for Visualization**

Export the aggregated tables into CSV files. These files are optimized for visualization, allowing us to build interactive dashboards for price monitoring.

In [27]:
# Exporting the final tables for visualization
global_timing.to_csv("out/mart_time_global.csv", index=False)
route_timing.to_csv("out/mart_route_lead_time.csv", index=False)
lead_time_price_pct.to_csv("out/mart_route_lead_time_price_pct.csv", index=False)
optimal_bucket.to_csv("out/mart_route_optimal_lead_time.csv", index=False)
dow_stats.to_csv("out/mart_daily_price_stats.csv", index=False)
dow_lt.to_csv("out/mart_daily_lead_time_price_stats.csv", index=False)
cheapest_day_per_route.to_csv("out/mart_route_best_days.csv", index=False)
best_day.to_csv("out/mart_final_recommendations.csv", index=False)

print("All analytics tables have been exported to the 'out/' folder.")

All analytics tables have been exported to the 'out/' folder.


In [28]:
# Download the files
#from google.colab import files
#files.download("out/mart_time_global.csv")
#files.download("out/mart_route_lead_time.csv")
#files.download("out/mart_route_lead_time_price_pct.csv")
#files.download("out/mart_route_optimal_lead_time.csv")
#files.download("out/looker_daily_price_stats.csv")
#files.download("out/looker_daily_lead_time_price_stats.csv")
#files.download("out/looker_route_best_days.csv")
#files.download("out/looker_final_recommendations.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>